# *SDS_Benchmark*: Landsat vs Sentinel-2 for CoastSat and SHOREX

(Work in progress)

Here we compare the accuracy of the SDS derived from Landsat and Sentinel-2 for CoastSat and SHOREX (the other algorithms are composites).

When running this notebook you will compare all the submitted time-series, the plots showing the comparisons and reporting the accuracy metrics are saved under **/evaluation/Landsat_vs_S2**. You can visualise each individual transect under **/evaluation/Landsat_vs_S2**

## Initial settings

To run this notebook you will need basic Python packages installed: `numpy`, `scipy`, `pandas`, `matplotlib`, `pytz`.

If you have [CoastSat](https://github.com/kvos/CoastSat) installed, you can activate that envrionment as it has all the necessary packages.

In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches
from matplotlib import gridspec
import colorcet as cc
from matplotlib.cm import get_cmap
import pandas as pd
from datetime import datetime, timedelta
from scipy import interpolate
from scipy import stats
import pytz
import json
import pdb
import string
# import utils module located in the repository (utils.py)
import utils
plt.ion()
letters = string.ascii_lowercase

# filepaths to all the datasets
fp_datasets = os.path.join(os.path.join(os.getcwd(),'datasets'))
names_datasets = os.listdir(fp_datasets)
names_datasets = [_ for _ in names_datasets if _ not in ['README.md','sites_info.txt','CALAMILLOR']]
print('\nBenchmark datasets available:\n%s'%(names_datasets))
                
# load site info dict if exists or create
fp_info = os.path.join(fp_datasets,'sites_info.txt')
if os.path.exists(fp_info):
    with open(fp_info,'r') as f: sites_info = json.load(f)  
    print('\nLoaded sites_info.txt.')
    for key in sites_info.keys(): print('%s: %s'%(key,sites_info[key]))

# filepaths to submissions
fp_submissions = os.path.join(os.path.join(os.getcwd(),'submissions'))
submissions = [_ for _ in os.listdir(fp_submissions) if _ not in ['README.md','example_submission']]
print('\nSubmitted shorelines: %s'%submissions)

# folder where to save all the figures and outputs
fp_eval =  os.path.join(os.getcwd(),'evaluation')
if not os.path.exists(fp_eval): os.makedirs(fp_eval)


Benchmark datasets available:
['DUCK', 'NARRABEEN', 'TORREYPINES', 'TRUCVERT']

Loaded sites_info.txt.
NARRABEEN: {'beach_slope': 0.1, 'contour_level': 0.7, 'epsg': 28356}
DUCK: {'beach_slope': 0.1, 'contour_level': 0.585, 'epsg': 32119}
TRUCVERT: {'beach_slope': 0.05, 'contour_level': 1.5, 'epsg': 32630}
TORREYPINES: {'beach_slope': 0.045, 'contour_level': 0.792, 'epsg': 26946}
CALAMILLOR: {'beach_slope': 0.1, 'contour_level': 0, 'epsg': 2062}

Submitted shorelines: ['team_CASSIE', 'team_CoastSat', 'team_Deltares', 'team_SHOREX', 'team_UQMAO']
